<a href="https://colab.research.google.com/github/daryoush/CudaAndFluxExperiments/blob/main/cudaexpOnJulia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
    "Random", "DataStructures", "Transformers","Statistics", "BenchmarkTools"\
    ]);' ###Pkg.precompile()'
!echo "DONE"

IndentationError: ignored

In [ ]:
[1,2,3]

In [ ]:
using CUDA

In [ ]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

if ENV["COLAB_GPU"] == "1"

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     417.434 ms (0.00% GC)
  median time:      431.325 ms (0.00% GC)
  mean time:        435.868 ms (0.73% GC)
  maximum time:     471.040 ms (0.71% GC)
  --------------
  samples:          12
  evals/sample:     1

In [ ]:
CUDA.memory_status()
GC.gc(true)
CUDA.memory_status()

Effective GPU memory usage: 34.08% (5.019 GiB/14.726 GiB)
CUDA allocator usage: 4.837 GiB
binned usage: 4.837 GiB (48.856 MiB allocated, 4.789 GiB cached)
